# Поиск гиперпараметров для каждой сети

## Задача

Нахождение наилучших гиперпараметров для CNN, LSTM, LSTM_CNN моделей для текстов полученных при помощи Word2Vec и FastText.

## Данные

Классы с архитектурами нейронных сетей и данные для обучения и теста взяты из файла *dataset_and_models.py*

## Расчёты

In [1]:
import numpy as np
import torch
from torch import nn, optim
from dataset_and_models import VECTOR_SIZE
from dataset_and_models import CNN, LSTM, LSTM_CNN
from dataset_and_models import w2v_data_train, w2v_data_test
from dataset_and_models import fasttext_data_train, fasttext_data_test
from sklearn.metrics import f1_score
from hyperopt import fmin, tpe, hp, space_eval
from functools import partial
import pickle

In [2]:
# установка устройва для расчётов
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [3]:
def train_evaluate(
        parametterization,
        model_class, 
        data_train,
        data_test,
        device=DEVICE,
        epochs=1
):
    """
    Функция для обучения и оценки модели
    :param parametterization: параметры модели и коэффициент скорости обучения
    :param model_class: класс модели нейронной сети
    :param data_train: данные для обучения
    :param data_test: данные для теста
    :param device: устройво для основных расчётов
    :param epochs: количество эпох обучения
    :return: (-1) * F1-score (weighted) 

             F1-score, т.к. типы отзывов распределены неравномерно, 
             (~60 % имеют положительную маркекровку, ~20 % - нейтральная и ~20 % негативных)
             и простая метрика как Точность будет менее информативна.

             Weighted, т.к. задача классификации имеет более чем 2 класса.

             Домножение на (-1) необходимо, так как нас интересует максимизация F1-score,
             а функция fmin может только минимизировать, вот и переворачиваем. 
    """
    # получение коэффициента скорости обучения
    lr = parametterization.pop('lr')
    
    # создание объекта модели нейронной сети
    model = model_class(**parametterization)
    model.to(device)

    # установка функции ошибки и оптимизатора
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # этап обучения модели
    for _ in range(epochs):
        for x, y in data_train:

            model.train()
            x, y = x.to(device), y.to(device)
            y_hat = model(x)

            optimizer.zero_grad()
            loss = loss_fn(y_hat, y)
            loss.backward()
            optimizer.step()

    # этап оценки модели
    model.eval()

    # список всех оценок для каждого батча
    f1_score_list = []

    with torch.no_grad():

        for x, y in data_test:
            x = x.to(device)
            
            f1_score_list.append(
                f1_score(
                    y_true=y.numpy(),
                    y_pred=model(x).argmax(axis=1).cpu().numpy(),
                    average='weighted'
                )
            )
        
    f1_score_list = np.array(f1_score_list)
    
    # освобождение памяти от модели
    del model
    torch.cuda.empty_cache()

    # среднее значение по всем оценкам батчей
    return (-1) * f1_score_list.mean()

In [4]:
def search_of_hyperparameters(
        search_parameters,
        model_class,
        data_train,
        data_test,
        number_of_try=100,
        epochs=1
):
    """
    Функция поиска гиперпараметров
    :param search_parameters: 
    :param model_class: класс модели нейронной сети
    :param data_train: данные для обучения
    :param data_test: данные для теста
    :param number_of_try: число итерация для алогоритма поиска
    :param epochs: число эпох для обучения одной сети
    :return: словарь содержащий найденные наилучшие гиперпараметры
    """
    best_hyperparameters = fmin(
        fn=partial(
            train_evaluate,
            model_class=model_class, 
            data_train=data_train,
            data_test=data_test,
            device=DEVICE,
            epochs=epochs
        ),
        space=search_parameters,
        algo=tpe.suggest,
        max_evals=number_of_try
    )
    
    print(space_eval(search_parameters, best_hyperparameters))

    return space_eval(search_parameters, best_hyperparameters)

In [5]:
# словарь для хранения наилучших гиперпараметров
best_hyperparameters = {}

### Подбор гиперпараметров для CNN

In [6]:
# область поиска гиперпараметров для CNN сети
cnn_parameters = {
    'lr': hp.uniform('lr', 1e-3, 1e-1),
    'number_of_filters': hp.choice('number_of_filters', range(10, 101, 5)),
    'vector_size': VECTOR_SIZE
}

In [7]:
# w2v CNN
best_hyperparameters['w2v_CNN'] = search_of_hyperparameters(
    search_parameters=cnn_parameters,
    model_class=CNN,
    data_train=w2v_data_train,
    data_test=w2v_data_test
)

100%|██████████| 100/100 [1:28:40<00:00, 53.20s/trial, best loss: -0.7177714674018713]
{'lr': 0.0019840863354355755, 'number_of_filters': 70, 'vector_size': 100}


In [8]:
# fasttesxt CNN
best_hyperparameters['fasttext_CNN'] = search_of_hyperparameters(
    search_parameters=cnn_parameters,
    model_class=CNN,
    data_train=fasttext_data_train,
    data_test=fasttext_data_test
)

100%|██████████| 100/100 [1:53:01<00:00, 67.81s/trial, best loss: -0.7167403494849508]
{'lr': 0.005238807869458511, 'number_of_filters': 75, 'vector_size': 100}


### Подбор гиперпараметров для LSTM

In [9]:
# область поиска гиперпараметров для LSTM сети
lstm_parameters = {
    'lr': hp.uniform('lr', 1e-3, 1e-1),
    'vector_size': VECTOR_SIZE,
    'lstm_out_features': hp.choice('lstm_out_features', range(25, int(VECTOR_SIZE * 0.6) + 1, 5))
}

In [10]:
# w2v LSTM
best_hyperparameters['w2v_LSTM'] = search_of_hyperparameters(
    search_parameters=lstm_parameters,
    model_class=LSTM,
    data_train=w2v_data_train,
    data_test=w2v_data_test
)

100%|██████████| 100/100 [1:26:45<00:00, 52.05s/trial, best loss: -0.6984205988593378]
{'lr': 0.013538336892539438, 'lstm_out_features': 50, 'vector_size': 100}


In [11]:
# fasttesxt LSTM
best_hyperparameters['fasttext_LSTM'] = search_of_hyperparameters(
    search_parameters=lstm_parameters,
    model_class=LSTM,
    data_train=fasttext_data_train,
    data_test=fasttext_data_test
)

100%|██████████| 100/100 [1:52:41<00:00, 67.61s/trial, best loss: -0.6760539322157916]
{'lr': 0.016816389472063677, 'lstm_out_features': 45, 'vector_size': 100}


### Подбор гиперпараметров для LSTM_CNN

In [12]:
# область поиска гиперпараметров для LSTM_CNN сети
lstm_cnn_parameters = {
    'lr': hp.uniform('lr', 1e-3, 1e-1),
    'vector_size': VECTOR_SIZE,
    'lstm_out': hp.choice('lstm_out', range(25, int(VECTOR_SIZE * 0.6) + 1, 5)),
    'number_of_filters': hp.choice('number_of_filters', range(10, 101, 5))
}

In [13]:
# w2v LSTM_CNN
best_hyperparameters['w2v_LSTM_CNN'] = search_of_hyperparameters(
    search_parameters=lstm_cnn_parameters,
    model_class=LSTM_CNN,
    data_train=w2v_data_train,
    data_test=w2v_data_test
)

100%|██████████| 100/100 [1:27:01<00:00, 52.21s/trial, best loss: -0.7161151110128887]
{'lr': 0.0010517228365822806, 'lstm_out': 55, 'number_of_filters': 80, 'vector_size': 100}


In [14]:
# fasttesxt LSTM_CNN
best_hyperparameters['fasttext_LSTM_CNN'] = search_of_hyperparameters(
    search_parameters=lstm_cnn_parameters,
    model_class=LSTM_CNN,
    data_train=fasttext_data_train,
    data_test=fasttext_data_test
)

100%|██████████| 100/100 [1:52:05<00:00, 67.25s/trial, best loss: -0.705023389704645]
{'lr': 0.003091052673013807, 'lstm_out': 30, 'number_of_filters': 20, 'vector_size': 100}


### Запись Гиперпараметров в файл

In [15]:
with open('data/best_hyperparameters.pkl', 'wb') as f:
    pickle.dump(best_hyperparameters, f)

## Результаты

Былы получены наиболее успешные гиперпараметры для 6 моделей: CNN, LSTM, LSTM_CNN обученных на Word2Vec и FastText текстах каждая.